# Import

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import datetime
import matplotlib.pyplot as plt
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import ndcg_score
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import tensorflow as tf
import tensorflow_ranking as tfr
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler



# Reduce File Size into chunks

In [ ]:
"""
# Load the large CSV file
file_path = 'C:/Users/98398/Desktop/P5/DM-AS2/Data/training_set_VU_DM.csv'
data = pd.read_csv(file_path)

# Define the size of each chunk
chunk_size = 500000  # This number can change 

# Split the CSV into chunks
for i in range(0, len(data), chunk_size):
    chunk = data.iloc[i:i + chunk_size]
    chunk.to_csv(f'chunk_{i//chunk_size}.csv', index=False)

"""

In [ ]:
df = pd.read_csv("chunk_0.csv")
df['date_time'] = pd.to_datetime(df['date_time'])
df.head()

In [ ]:
print(len(df.columns))

df.columns

# Data cleaning

要增加的部分

1，如果一个人只浏览但没有点击，那么它的数据在排序中是否是无意义的？

## Remove missing values

In [ ]:
file_path = 'C:/Users/98398/Desktop/P5/DM-AS2/Test_data/chunk_0_cleaned.csv'

df_cleaned_chunk0 = pd.read_csv(file_path)

In [ ]:
print(df_cleaned_chunk0.head())

## Classify according to different variable types

In [ ]:
Criteria_in_the_search_query = ['date_time', 'site_id','srch_destination_id', 'srch_length_of_stay', 'srch_booking_window',
       'srch_adults_count', 'srch_children_count', 'srch_room_count',
       'srch_saturday_night_bool']

Static_hotel_characteristics = ['prop_country_id',
       'prop_id', 'prop_starrating', 'prop_review_score', 'prop_brand_bool',
       'prop_location_score1', 'prop_location_score2',
       'prop_log_historical_price','srch_query_affinity_score','orig_destination_distance']

Dynamic_hotel_characteristics = ['random_bool', 'position', 'price_usd', 'promotion_flag','click_bool', 'gross_bookings_usd',
       'booking_bool']

Visitor_information = ['srch_id', 'visitor_location_country_id',
       'visitor_hist_starrating', 'visitor_hist_adr_usd']

Competitive_information = ['comp1_rate', 'comp1_inv',
       'comp1_rate_percent_diff', 'comp2_rate', 'comp2_inv',
       'comp2_rate_percent_diff', 'comp3_rate', 'comp3_inv',
       'comp3_rate_percent_diff', 'comp4_rate', 'comp4_inv',
       'comp4_rate_percent_diff', 'comp5_rate', 'comp5_inv',
       'comp5_rate_percent_diff', 'comp6_rate', 'comp6_inv',
       'comp6_rate_percent_diff', 'comp7_rate', 'comp7_inv',
       'comp7_rate_percent_diff', 'comp8_rate', 'comp8_inv',
       'comp8_rate_percent_diff']

Other = []

all_catogories = [Criteria_in_the_search_query, Static_hotel_characteristics, 
                 Dynamic_hotel_characteristics, Visitor_information,Competitive_information,Other]
all_variables = []
for i in all_catogories:
    for j in i:
        all_variables.append(j)

df_variables = df.columns.to_list()

differences = set(all_variables).symmetric_difference(set(df_variables))

if differences == set():
    print('All catogorized')
else:
    print(f'Differences between df_variables and catogorized variables: {differences}')


### Criteria_in_the_search_query

In [ ]:
print(Criteria_in_the_search_query)

# Model lambdarank

## Basic model(without parameter choose and feature engineering)

In [ ]:
# 增加一个计算得分的列
def assign_scores(row):
    if row['booking_bool'] == 1:
        return 5
    elif row['click_bool'] == 1:
        return 1
    else:
        return 0

In [ ]:
file_path = 'C:/Users/98398/Desktop/P5/DM-AS2/Test_data/chunk_0_cleaned.csv'

#file_path = 'C:/Users/98398/Desktop/P5/DM-AS2/Test_data/cleaned_training_set_VU_DM.csv'
#file_path = 'C:/Users/98398/Desktop/P5/DM-AS2/Test_data/training_set_VU_DM.csv'
file_path = 'C:/Users/98398/Desktop/P5/DM-AS2/Kaggle/train.csv'
df = pd.read_csv(file_path)

df['score'] = df.apply(assign_scores, axis=1)

features = ['site_id','srch_destination_id', 'srch_length_of_stay', 'srch_booking_window',
       'srch_adults_count', 'srch_children_count', 'srch_room_count',
       'srch_saturday_night_bool','prop_country_id',
       'prop_id', 'prop_starrating', 'prop_review_score', 'prop_brand_bool',
       'prop_location_score1', 'prop_location_score2',
       'prop_log_historical_price',

       'srch_query_affinity_score','orig_destination_distance','random_bool', 
       'price_usd', 'promotion_flag','srch_id', 'visitor_location_country_id',
       'visitor_hist_starrating', 'visitor_hist_adr_usd','comp1_rate', 

       'comp1_inv',
       'comp1_rate_percent_diff', 'comp2_rate', 'comp2_inv',
       'comp2_rate_percent_diff', 'comp3_rate', 'comp3_inv',
       'comp3_rate_percent_diff', 'comp4_rate', 'comp4_inv',
       'comp4_rate_percent_diff', 'comp5_rate', 'comp5_inv',
       'comp5_rate_percent_diff', 'comp6_rate', 'comp6_inv',
       'comp6_rate_percent_diff', 'comp7_rate', 'comp7_inv',
       'comp7_rate_percent_diff', 'comp8_rate', 'comp8_inv',
       'comp8_rate_percent_diff']

# 划分数据集
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['srch_id'])

# 准备 LightGBM 数据结构
train_data = lgb.Dataset(train_df[features], label=train_df['score'], group=train_df['srch_id'].value_counts().sort_index())
test_data = lgb.Dataset(test_df[features], label=test_df['score'], group=test_df['srch_id'].value_counts().sort_index())

# 设置模型参数
params = {
    'objective': 'lambdarank',
    'metric': 'ndcg',
    'ndcg_eval_at': [3, 5],
    'learning_rate': 0.1,
    'num_leaves': 31,
    'verbose': -1
}

# 训练模型
num_round = 100
bst = lgb.train(params, train_data, num_round, valid_sets=[test_data])

# 预测测试集
test_pred = bst.predict(test_df[features])

# 评估模型，计算 NDCG 分数
test_df['predictions'] = test_pred

# 首先确保数据按照 srch_id 和 predictions 降序排序
test_df.sort_values(['srch_id', 'predictions'], ascending=[True, False], inplace=True)

# 分组并计算每个搜索会话的 NDCG
grouped = test_df.groupby('srch_id')
ndcg_scores = []

for name, group in grouped:
    group = group.sort_values('predictions', ascending=False)
    true_relevance = group['score'].values
    scores_pred = group['predictions'].values
    # 计算当前搜索会话的 NDCG 分数，并追加到列表中
    if len(np.unique(true_relevance)) > 1:  # 只计算有正样本的会话
        ndcg_scores.append(ndcg_score([true_relevance], [scores_pred], k=5))

average_ndcg = np.mean(ndcg_scores)
print(f"Average NDCG Score: {average_ndcg}")


### Result output

In [ ]:
# 使用迭代器逐块读取数据
chunk_size = 10000
reader = pd.read_csv('C:/Users/98398/Desktop/P5/DM-AS2/Data/test_set_VU_DM.csv', chunksize=chunk_size)

predictions = []  # 创建一个空列表以存储每个块的预测结果
for chunk in reader:
    # 可以在这里添加数据预处理步骤，比如填充缺失值等
    chunk_pred = bst.predict(chunk[features])  # 应用模型进行预测
    chunk['predictions'] = chunk_pred  # 将预测结果添加到 DataFrame
    predictions.append(chunk[['srch_id', 'prop_id', 'predictions']])  # 仅保留需要的列

# 合并所有批次的预测结果
final_predictions = pd.concat(predictions)

In [ ]:
# 确保按照预测分数排序，如果 Kaggle 要求
final_predictions.sort_values(['srch_id', 'predictions'], ascending=[True, False], inplace=True)

# 选择需要的列
final_predictions = final_predictions[['srch_id', 'prop_id']]

# 保存为 CSV 文件，确保不包含索引，包含列标题
final_predictions.to_csv('train=all_cleaned.csv', index=False, header=True)

In [ ]:
print(final_predictions)

### All train data

In [ ]:
All_test_df = pd.read_csv('C:/Users/98398/Desktop/P5/DM-AS2/Data/training_set_VU_DM.csv')

# 预测测试集

All_test_pred = bst.predict(All_test_df[features])

# 评估模型，计算 NDCG 分数
All_test_df['predictions'] = All_test_pred

All_test_df['score'] = All_test_df.apply(assign_scores, axis=1)
# 假设 df 是你的 DataFrame
# 首先确保数据按照 srch_id 和 predictions 降序排序
All_test_df.sort_values(['srch_id', 'predictions'], ascending=[True, False], inplace=True)

# 分组并计算每个搜索会话的 NDCG
grouped = All_test_df.groupby('srch_id')
ndcg_scores = []

for name, group in grouped:
    # 确保每组数据中的预测是降序的（虽然已全局排序，这里是双重确认）
    group = group.sort_values('predictions', ascending=False)
    true_relevance = group['score'].values
    scores_pred = group['predictions'].values
    # 计算当前搜索会话的 NDCG 分数，并追加到列表中
    if len(np.unique(true_relevance)) > 1:  # 只计算有正样本的会话
        ndcg_scores.append(ndcg_score([true_relevance], [scores_pred], k=5))

# 计算平均 NDCG 分数
average_ndcg = np.mean(ndcg_scores)
print(f"Average NDCG Score: {average_ndcg}")


## Use all cleaned data without feature engineering

## With feature engineering by Menghan 

### Model build

In [ ]:
file_path = 'C:/Users/98398/Desktop/P5/DM-AS2/Test_data/feature_engineered_training_chunk_0.csv'

#file_path = 'C:/Users/98398/Desktop/P5/DM-AS2/Test_data/cleaned_training_set_VU_DM.csv'
#file_path = 'C:/Users/98398/Desktop/P5/DM-AS2/Test_data/training_set_VU_DM.csv'
df = pd.read_csv(file_path)

# 读取CSV文件的前100行
df1 = pd.read_csv('C:/Users/98398/Desktop/P5/DM-AS2/Test_data/feature_engineered_training_chunk_0.csv', nrows=100)

# 将这100行保存到一个新的CSV文件中
df1.to_csv('feature_engineered_training_chunk_0_100.csv',index=False)

df.head()


In [ ]:
# 增加一个计算得分的列
def assign_scores(row):
    if row['booking_bool'] == 1:
        return 5
    elif row['click_bool'] == 1:
        return 1
    else:
        return 0

df['score'] = df.apply(assign_scores, axis=1)


In [ ]:
columns = df.columns

# 使用列表推导式筛选出不需要删除的列
features = [
    col for col in columns if col not in ['date_time', 'position', 'click_bool', 'booking_bool', 'score']
    and 'gross_bookings_usd' not in col and 'position' not in col
]

print(features)

print(len(features))
print(len(df.columns))

In [ ]:
# 划分数据集
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['srch_id'])

# 准备 LightGBM 数据结构
train_data = lgb.Dataset(train_df[features], label=train_df['score'], group=train_df['srch_id'].value_counts().sort_index())
test_data = lgb.Dataset(test_df[features], label=test_df['score'], group=test_df['srch_id'].value_counts().sort_index())

# 设置模型参数
params = {
    'objective': 'lambdarank',
    'metric': 'ndcg',
    'ndcg_eval_at': [3, 5],
    'learning_rate': 0.1,
    'num_leaves': 31,
    'verbose': -1
}

# 训练模型
num_round = 100
bst = lgb.train(params, train_data, num_round, valid_sets=[test_data])

# 预测测试集
test_pred = bst.predict(test_df[features])

# 评估模型，计算 NDCG 分数
test_df['predictions'] = test_pred

# 首先确保数据按照 srch_id 和 predictions 降序排序
test_df.sort_values(['srch_id', 'predictions'], ascending=[True, False], inplace=True)

# 分组并计算每个搜索会话的 NDCG
grouped = test_df.groupby('srch_id')
ndcg_scores = []

for name, group in grouped:
    group = group.sort_values('predictions', ascending=False)
    true_relevance = group['score'].values
    scores_pred = group['predictions'].values
    # 计算当前搜索会话的 NDCG 分数，并追加到列表中
    if len(np.unique(true_relevance)) > 1:  # 只计算有正样本的会话
        ndcg_scores.append(ndcg_score([true_relevance], [scores_pred], k=5))

average_ndcg = np.mean(ndcg_scores)
print(f"Average NDCG Score: {average_ndcg}")


### Result output

In [ ]:
# 使用迭代器逐块读取数据
chunk_size = 10000
reader = pd.read_csv('C:/Users/98398/Desktop/P5/DM-AS2/Data/test_set_VU_DM.csv', chunksize=chunk_size)

predictions = []  # 创建一个空列表以存储每个块的预测结果
for chunk in reader:
    # 可以在这里添加数据预处理步骤，比如填充缺失值等
    chunk_pred = bst.predict(chunk[features])  # 应用模型进行预测
    chunk['predictions'] = chunk_pred  # 将预测结果添加到 DataFrame
    predictions.append(chunk[['srch_id', 'prop_id', 'predictions']])  # 仅保留需要的列

# 合并所有批次的预测结果
final_predictions = pd.concat(predictions)

In [ ]:
# 确保按照预测分数排序，如果 Kaggle 要求
final_predictions.sort_values(['srch_id', 'predictions'], ascending=[True, False], inplace=True)

# 选择需要的列
final_predictions = final_predictions[['srch_id', 'prop_id']]

# 保存为 CSV 文件，确保不包含索引，包含列标题
final_predictions.to_csv('train=all_cleaned.csv', index=False, header=True)

In [ ]:
print(final_predictions)

# Model DNN

## #Final2

In [ ]:
def ndcg_loss(y_true, y_pred):
    # 假定y_true和y_pred的形状都是(batch_size, 1)
    # 计算IDCG（理想情况下的NDCG分母）
    sorted_true = tf.sort(y_true, direction='DESCENDING')
    idcg = tf.math.reduce_sum((2 ** sorted_true - 1) / tf.math.log2(tf.range(2, tf.size(sorted_true) + 2, dtype=tf.float32)))

    # 对预测值进行排序
    _, indices = tf.nn.top_k(y_pred, k=tf.shape(y_pred)[0])
    sorted_preds = tf.gather(y_true, indices)
    
    # 计算DCG
    dcg = tf.math.reduce_sum((2 ** sorted_preds - 1) / tf.math.log2(tf.range(2, tf.size(sorted_preds) + 2, dtype=tf.float32)))
    
    # 计算NDCG
    ndcg = dcg / idcg
    # 损失是1-NDCG
    return 1.0 - ndcg

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import tensorflow as tf


# 文件路径和块大小
file_path = 'C:/Users/98398/Desktop/P5/DM-AS2/Test_data/feature_engineered_training_set_VU_DM.csv'
chunk_size = 200000  # 根据可用内存调整大小

# 读取数据并计算总块数
total_chunks = sum(1 for _ in pd.read_csv(file_path, chunksize=chunk_size))
train_chunk_count = int(total_chunks * 0.8)  # 用于训练的块的数量

# 定义分数赋值函数
def assign_scores(row):
    if row['booking_bool'] == 1:
        return 5
    elif row['click_bool'] == 1:
        return 1
    else:
        return 0

# 类别映射
mapping = {0: 0, 1: 1, 5: 2}

# 初始化标准化处理器
scaler = StandardScaler()
model_defined = False
model = None

# 读取数据块，进行训练和测试
for i, chunk in enumerate(pd.read_csv(file_path, chunksize=chunk_size)):
    chunk['score'] = chunk.apply(assign_scores, axis=1).map(mapping)  # 应用评分函数并映射类别
    chunk_features = [col for col in chunk.columns if col not in ['date_time', 'position', 'click_bool', 'booking_bool', 'score', 'gross_bookings_usd', 'srch_id']]

    # 标准化数据
    if not model_defined:
        scaler.fit(chunk[chunk_features])  # 只在第一个块上拟合scaler
        model_defined = True
    
    chunk[chunk_features] = scaler.transform(chunk[chunk_features])

    if i < train_chunk_count:  # 训练数据
        if model is None:
            # 定义模型
            model = Sequential([
                Dense(128, activation='relu', input_shape=(len(chunk_features),)),
                Dropout(0.3),
                Dense(64, activation='relu'),
                Dropout(0.3),
                Dense(1, activation='sigmoid')  # 假设为回归任务
            ])
            model.compile(optimizer='adam', loss=ndcg_loss)

        # 训练模型
        X_train = chunk[chunk_features]
        y_train = chunk['score']
        model.fit(X_train, y_train, epochs=1, batch_size=32)
    else:  # 测试数据
        X_test = chunk[chunk_features]
        y_test = chunk['score']
        predictions = model.predict(X_test)
        chunk['predictions'] = predictions.flatten()

        # 计算NDCG得分
        chunk = chunk.sort_values(['srch_id', 'predictions'], ascending=[True, False])
        grouped = chunk.groupby('srch_id')
        ndcg_scores = [ndcg_score([group['score'].tolist()], [group['predictions'].tolist()]) for _, group in grouped if len(np.unique(group['score'])) > 1]
        average_ndcg = np.mean(ndcg_scores)
        print(f"Average NDCG Score: {average_ndcg}")


## Final(using loss='sparse_categorical_crossentropy') Very bad.

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import ndcg_score

file_path = 'C:/Users/98398/Desktop/P5/DM-AS2/Test_data/feature_engineered_training_set_VU_DM.csv'
chunk_size = 200000  # 根据可用内存调整大小
total_chunks = sum(1 for _ in pd.read_csv(file_path, chunksize=chunk_size))  # 计算总块数
train_chunk_count = int(total_chunks * 0.8)  # 用于训练的块的数量

In [ ]:
# 定义分数赋值函数
def assign_scores(row):
    if row['booking_bool'] == 1:
        return 5
    elif row['click_bool'] == 1:
        return 1
    else:
        return 0

# 标准化处理器初始化
scaler = StandardScaler()
model_defined = False
model = None

# 读取数据块，训练和测试
for i, chunk in enumerate(pd.read_csv(file_path, chunksize=chunk_size)):
    chunk['score'] = chunk.apply(assign_scores, axis=1) # 应用评分函数并映射类别
    chunk_features = [col for col in chunk.columns if col not in ['date_time', 'position', 'click_bool', 'booking_bool', 'score', 'gross_bookings_usd', 'srch_id']]
    #if i < train_chunk_count:  # 训练数据
    if i < 10:
        if not model_defined:
            model = Sequential([
                Dense(128, activation='relu', input_shape=(len(chunk_features),)),
                Dropout(0.3),
                Dense(64, activation='relu'),
                Dropout(0.3),
                Dense(1, activation='linear')
            ])
            model.compile(optimizer='adam',
              loss='mean_squared_error',  # 使用 MSE 作为损失函数
              metrics=['mean_squared_error'])  # 评估指标使用
            model_defined = True
        # 标准化训练数据
        scaler.partial_fit(chunk[chunk_features])  # 计算标准化参数
        X_train = scaler.transform(chunk[chunk_features])
        y_train = chunk['score'].values
        train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(32)
        model.fit(train_dataset, epochs=1)
    if i == 11 or i == 12:    
    #else:  # 测试数据
        X_test = scaler.transform(chunk[chunk_features])
        y_test = chunk['score'].values
        test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(32)
        predictions = model.predict(test_dataset)
        chunk['predictions'] = np.argmax(predictions, axis=1)  # 获取概率最高的类别

        # 按 srch_id 和 predictions 降序排序
        chunk.sort_values(['srch_id', 'predictions'], ascending=[True, False], inplace=True)

        # 计算 NDCG 分数
        grouped = chunk.groupby('srch_id')
        ndcg_scores = []
        for name, group in grouped:
            true_relevance = group['score'].values
            scores_pred = group['predictions'].values
            if len(np.unique(true_relevance)) > 1:
                ndcg_scores.append(ndcg_score([true_relevance], [scores_pred], k=5))

        average_ndcg = np.mean(ndcg_scores)
        print(f"Average NDCG Score: {average_ndcg}")

## Test

In [ ]:
def assign_scores(row):
    if row['booking_bool'] == 1:
        return 5
    elif row['click_bool'] == 1:
        return 1
    else:
        return 0

In [ ]:
file_path = 'C:/Users/98398/Desktop/P5/DM-AS2/Test_data/feature_engineered_training_set_VU_DM.csv'
chunk_size = 100000  # 根据可用内存调整大小


# 首次读取数据，只为获取全部的 srch_id 以进行分层抽样
srch_ids = pd.Series(dtype=int)
for chunk in pd.read_csv(file_path, chunksize=chunk_size, usecols=['srch_id']):
    srch_ids = pd.concat([srch_ids, chunk['srch_id']])
srch_id_counts = srch_ids.value_counts()
train_ids, test_ids = train_test_split(srch_id_counts.index, test_size=0.2, random_state=42)

# 初始化数据框以存储训练和测试数据
train_df = pd.DataFrame()
test_df = pd.DataFrame()

# 读取数据并根据划分的srch_id分到训练集或测试集中
for chunk in pd.read_csv(file_path, chunksize=chunk_size):
    chunk['score'] = chunk.apply(assign_scores, axis=1)  # 应用评分函数
    train_chunk = chunk[chunk['srch_id'].isin(train_ids)]
    test_chunk = chunk[chunk['srch_id'].isin(test_ids)]
    train_df = pd.concat([train_df, train_chunk])
    test_df = pd.concat([test_df, test_chunk])

In [ ]:
# 特征选择
features = [col for col in train_df.columns if col not in ['srch_id', 'date_time', 'position', 'click_bool', 'booking_bool', 'score'] and 'gross_bookings_usd' not in col]

# 标准化特征
scaler = StandardScaler()
train_df[features] = scaler.fit_transform(train_df[features])
test_df[features] = scaler.transform(test_df[features])

In [ ]:
mapping = {0: 0, 1: 1, 5: 2}
train_df['score'] = train_df['score'].map(mapping)
test_df['score'] = test_df['score'].map(mapping)

# 模型构建
model = Sequential([
    Dense(128, activation='relu', input_shape=(len(features),)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(3, activation='softmax')  # 更改为3，因为现在有三个类别
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 训练模型
train_dataset = tf.data.Dataset.from_tensor_slices((train_df[features].values, train_df['score'].values)).batch(32)
model.fit(train_dataset, epochs=10)


In [ ]:
# 预测
test_dataset = tf.data.Dataset.from_tensor_slices((test_df[features].values, test_df['score'].values)).batch(32)
test_df['predictions'] = model.predict(test_dataset).flatten()

# 按 srch_id 和 predictions 降序排序
test_df.sort_values(['srch_id', 'predictions'], ascending=[True, False], inplace=True)

# 计算 NDCG 分数
grouped = test_df.groupby('srch_id')
ndcg_scores = []
for name, group in grouped:
    true_relevance = group['score'].values
    scores_pred = group['predictions'].values
    if len(np.unique(true_relevance)) > 1:
        ndcg_scores.append(ndcg_score([true_relevance], [scores_pred], k=5))

average_ndcg = np.mean(ndcg_scores)
print(f"Average NDCG Score: {average_ndcg}")

## Other

In [ ]:
file_path = 'C:/Users/98398/Desktop/P5/DM-AS2/Test_data/feature_engineered_training_chunk_0.csv'
file_path = 'C:/Users/98398/Desktop/P5/DM-AS2/Test_data/feature_engineered_training_set_VU_DM.csv'
#file_path = 'C:/Users/98398/Desktop/P5/DM-AS2/Test_data/cleaned_training_set_VU_DM.csv'
#file_path = 'C:/Users/98398/Desktop/P5/DM-AS2/Test_data/training_set_VU_DM.csv'
df = pd.read_csv(file_path)

df.head()

In [ ]:
df['score'] = df.apply(assign_scores, axis=1)
columns = df.columns

# 使用列表推导式筛选出不需要删除的列
features = [
    col for col in columns if col not in ['date_time', 'position', 'click_bool', 'booking_bool', 'score']
    and 'gross_bookings_usd' not in col and 'position' not in col
]

## Data preparation and preprocessing

In [ ]:
# 数据划分
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['srch_id'])

# 特征标准化
scaler = StandardScaler()
train_df[features] = scaler.fit_transform(train_df[features])
test_df[features] = scaler.transform(test_df[features])

# 创建 TensorFlow 数据集
def make_dataset(data_df):
    dataset = tf.data.Dataset.from_tensor_slices((data_df[features].values, data_df['score'].values))
    dataset = dataset.shuffle(buffer_size=len(data_df)).batch(32).repeat()
    return dataset

train_dataset = make_dataset(train_df)
test_dataset = make_dataset(test_df)


## Model build

In [ ]:
# 构建模型
model = Sequential([
    Dense(128, activation='relu', input_shape=(len(features),)),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1)  # 输出层，无激活函数
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# 模型训练
model.fit(train_dataset, epochs=10, steps_per_epoch=200, validation_data=test_dataset, validation_steps=50)


## predict and evaluate

In [ ]:
# 预测测试集
test_df['predictions'] = model.predict(test_df[features])

# 按 srch_id 和 predictions 降序排序
test_df.sort_values(['srch_id', 'predictions'], ascending=[True, False], inplace=True)

# 分组并计算每个搜索会话的 NDCG
grouped = test_df.groupby('srch_id')
ndcg_scores = []

for name, group in grouped:
    true_relevance = group['score'].values
    scores_pred = group['predictions'].values
    if len(np.unique(true_relevance)) > 1:  # 只计算有正样本的会话
        ndcg_scores.append(ndcg_score([true_relevance], [scores_pred], k=5))

average_ndcg = np.mean(ndcg_scores)
print(f"Average NDCG Score: {average_ndcg}")


# Model NN using TensorFlow Ranking

In [ ]:
def assign_scores(row):
    if row['booking_bool'] == 1:
        return 5
    elif row['click_bool'] == 1:
        return 1
    else:
        return 0

In [ ]:
# 逐个读取feature_engineered_training_chunk{i}并上下拼接到一个df
base_path = 'D:/Table/P5/DM-AS2/Test_data/training/'
file_pattern = 'best_feature_engineered_training_chunk_{}.csv'
for i in range(2):
    df_chunk = pd.read_csv(base_path + file_pattern.format(i))
    df_chunk['score'] = df_chunk.apply(assign_scores, axis=1)
    if i == 0:
        df = df_chunk
    else:
        df = pd.concat([df, df_chunk], axis=0)
df.head()

In [ ]:
feature_columns = [col for col in df.columns if col not in ['srch_id', 'date_time', 'position', 'click_bool', 'booking_bool', 'score', 'gross_bookings_usd']]

# 标准化特征
unique_ids = df['srch_id'].unique()
train_ids, test_ids = train_test_split(unique_ids, test_size=0.2, random_state=42, shuffle=True)
train_data = df[df['srch_id'].isin(train_ids)]
test_data = df[df['srch_id'].isin(test_ids)]

scaler = StandardScaler()
train_data[feature_columns] = scaler.fit_transform(train_data[feature_columns])
test_data[feature_columns] = scaler.fit_transform(test_data[feature_columns])

In [ ]:
# 1. 逐个读取feature_engineered_training_chunk{i}并上下拼接到一个df
base_path = 'D:/Table/P5/DM-AS2/Test_data/training/'
file_pattern = 'best_feature_engineered_training_chunk_{}.csv'
for i in range(2):
    df_chunk = pd.read_csv(base_path + file_pattern.format(i))
    df_chunk['score'] = df_chunk.apply(assign_scores, axis=1)
    if i == 0:
        data = df_chunk
    if i == 1:
        test = df_chunk
    else:
        data = pd.concat([data, df_chunk], axis=0)

# 2. 数据预处理
# 定义得分
feature_columns = [col for col in data.columns if col not in ['srch_id', 'date_time', 'position', 'click_bool', 'booking_bool', 'score', 'gross_bookings_usd']]

'''
unique_ids = data['srch_id'].unique()
train_ids, test_ids = train_test_split(unique_ids, test_size=0.2, random_state=42)
#train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['srch_id'])
train_data = data[data['srch_id'].isin(train_ids)]
test_data = data[data['srch_id'].isin(test_ids)]
'''
scaler = StandardScaler()
data[feature_columns] = scaler.fit_transform(data[feature_columns])

train_data = data
test_data = test

# 3. 构建按 srch_id 分组的测试数据
grouped_test_data = test_data.groupby('srch_id').apply(
    lambda x: x.sort_values(by='score', ascending=False)
).reset_index(drop=True)

# 4. 创建 TensorFlow Ranking 模型输入函数
def input_fn(data, batch_size=32):
    def _fn():
        # Prepare features as a dictionary mapping from feature names to tensors.
        # Ensure these names and structures match what the model expects.
        features = {k: tf.reshape(tf.convert_to_tensor(data[k].values), (-1, 1)) for k in feature_columns}
        labels = tf.cast(tf.convert_to_tensor(data['score'].values), tf.float32)
        #print("Labels dtype:", labels.dtype)
        # Batch the data
        dataset = tf.data.Dataset.from_tensor_slices((features, labels))
        dataset = dataset.batch(batch_size)
        #print(dataset)
        return dataset
    return _fn

# 5. 定义模型

optimizer = tf.keras.optimizers.Adagrad(learning_rate=0.1)
loss = tfr.keras.losses.get(tfr.losses.RankingLossKey.APPROX_NDCG_LOSS)
model = tfr.keras.model.create_keras_model(
    network=tfr.keras.canned.DNNRankingNetwork(
        context_feature_columns=None,
        example_feature_columns={col: tf.feature_column.numeric_column(col) for col in feature_columns},
        hidden_layer_dims=[128, 64],
        activation=tf.nn.relu,
        dropout=0.5),
    loss=loss,
    metrics=[tfr.keras.metrics.NDCGMetric(name="ndcg_metric")],
    optimizer=optimizer,
    size_feature_name=None)

# 6. 训练模型
train_input_fn = input_fn(train_data)
model.fit(train_input_fn(), steps_per_epoch=1000, epochs = 10)

# 7. 评估模型 - 计算按 srch_id 分组的 NDCG
test_input_fn = input_fn(grouped_test_data)
predictions = model.predict(test_input_fn())

# 将预测与真实得分按 srch_id 添加到数据帧，然后计算 NDCG
grouped_test_data['predictions'] = predictions.flatten()
grouped = grouped_test_data.groupby('srch_id')
ndcg_scores = []
for name, group in grouped:
    true_labels = group['score'].tolist()
    pred_labels = group['predictions'].tolist()
    # 计算当前搜索会话的 NDCG 分数，并追加到列表中
    if len(np.unique(true_labels)) > 1:  # 只计算有正样本的会话
        ndcg_scores.append(ndcg_score([true_labels], [pred_labels], k=5))

average_ndcg = np.mean(ndcg_scores)
print(f"Average NDCG Score across groups: {average_ndcg}")


In [ ]:

scaler = StandardScaler()
train_data[feature_columns] = scaler.fit_transform(train_data[feature_columns])
optimizer = tf.keras.optimizers.Adagrad(learning_rate=0.1)
loss = tfr.keras.losses.get(tfr.losses.RankingLossKey.APPROX_NDCG_LOSS)
model = tfr.keras.model.create_keras_model(
    network=tfr.keras.canned.DNNRankingNetwork(
        context_feature_columns=None,
        example_feature_columns={col: tf.feature_column.numeric_column(col) for col in feature_columns},
        hidden_layer_dims=[128, 64],
        activation=tf.nn.relu,
        dropout=0.1),
    loss=loss,
    metrics=[tfr.keras.metrics.NDCGMetric(name="ndcg_metric")],
    optimizer=optimizer,
    size_feature_name=None)
# 6. 训练模型
train_input_fn = input_fn(train_data)
model.fit(train_input_fn(), steps_per_epoch=1000, epochs=10)
print(1)
# 7. 评估模型 - 计算按 srch_id 分组的 NDCG
test_input_fn = input_fn(grouped_test_data)
predictions = model.predict(test_input_fn())

# 将预测与真实得分按 srch_id 添加到数据帧，然后计算 NDCG
grouped_test_data['predictions'] = predictions.flatten()
grouped = grouped_test_data.groupby('srch_id')
ndcg_scores = []
for name, group in grouped:
    true_labels = group['score'].tolist()
    pred_labels = group['predictions'].tolist()
    # 计算当前搜索会话的 NDCG 分数，并追加到列表中
    if len(np.unique(true_labels)) > 1:  # 只计算有正样本的会话
        ndcg_scores.append(ndcg_score([true_labels], [pred_labels], k=5))

average_ndcg = np.mean(ndcg_scores)
print(f"Average NDCG Score across groups: {average_ndcg}")

# Model RF

In [ ]:
def assign_scores(row):
    if row['booking_bool'] == 1:
        return int(5)
    elif row['click_bool'] == 1:
        return int(1)
    else:
        return int(0)

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

# 数据文件路径
base_path = 'D:/Table/P5/DM-AS2/Test_data/training/'
file_pattern = 'best_feature_engineered_training_chunk_{}.csv'

df = pd.read_csv(base_path + file_pattern.format(0), nrows = 50000)
    

print(0)
# 数据预处理，定义分数
df['score'] = df.apply(lambda row: 5 if row['booking_bool'] == 1 else 1 if row['click_bool'] == 1 else 0, axis=1)
exclude_columns = ['date_time', 'position', 'click_bool', 'booking_bool', 'score', 'gross_bookings_usd']
features = [col for col in df.columns if col not in exclude_columns]
print(1)
# 分割数据集
X_train, X_test, y_train, y_test, srch_id_train, srch_id_test = train_test_split(df[features], df['score'], df['srch_id'], test_size=0.20, random_state=42)
print(2)
# 标准化数据
#scaler = StandardScaler()
#X_train_scaled = scaler.fit_transform(X_train)
#X_test_scaled = scaler.transform(X_test)

# 初始化模型
model = RandomForestRegressor(n_estimators=50, random_state=42)

# 训练模型
#model.fit(X_train_scaled, y_train)
model.fit(X_train, y_train)
print(3)
# 进行预测
#predictions = model.predict(X_test_scaled)
predictions = model.predict(X_test)

# 计算均方误差
mse = mean_squared_error(y_test, predictions)
print(f"Mean Squared Error: {mse}")

# 计算 NDCG
# 根据 srch_id 分组并计算每个组的 NDCG
df_test = pd.DataFrame({'srch_id': srch_id_test, 'y_test': y_test, 'predictions': predictions})
ndcg_scores = []

# 分组并为每个搜索 ID 排序预测结果
for name, group in df_test.groupby('srch_id'):
    true_relevance = group.sort_values('predictions', ascending=False)['y_test'].tolist()
    scores_pred = group.sort_values('predictions', ascending=False)['predictions'].tolist()
    if len(np.unique(true_relevance)) > 1:  # 只计算有正样本的会话
        ndcg_scores.append(ndcg_score([true_relevance], [scores_pred], k=5))

# 计算平均 NDCG 分数
average_ndcg = sum(ndcg_scores) / len(ndcg_scores)
print(f"Average NDCG Score: {average_ndcg}")

In [ ]:
base_path = 'D:/Table/P5/DM-AS2/Test_data/training/'
file_pattern = 'best_feature_engineered_training_chunk_{}.csv'

df = pd.read_csv(base_path + file_pattern.format(8), nrows = 200000)
df['score'] = df.apply(lambda row: 5 if row['booking_bool'] == 1 else 1 if row['click_bool'] == 1 else 0, axis=1)

X_train, X_test, y_train, y_test, srch_id_train, srch_id_test = train_test_split(df[features], df['score'], df['srch_id'], test_size=0.20, random_state=42)

predictions = model.predict(X_test)

# 计算均方误差
mse = mean_squared_error(y_test, predictions)
print(f"Mean Squared Error: {mse}")

# 计算 NDCG
# 根据 srch_id 分组并计算每个组的 NDCG
df_test = pd.DataFrame({'srch_id': srch_id_test, 'y_test': y_test, 'predictions': predictions})
ndcg_scores = []

# 分组并为每个搜索 ID 排序预测结果
for name, group in df_test.groupby('srch_id'):
    true_relevance = group.sort_values('predictions', ascending=False)['y_test'].tolist()
    scores_pred = group.sort_values('predictions', ascending=False)['predictions'].tolist()
    if len(np.unique(true_relevance)) > 1:  # 只计算有正样本的会话
        ndcg_scores.append(ndcg_score([true_relevance], [scores_pred], k=5))

# 计算平均 NDCG 分数
average_ndcg = sum(ndcg_scores) / len(ndcg_scores)
print(f"Average NDCG Score: {average_ndcg}")

## 结果输出

In [ ]:
# 使用迭代器逐块读取数据
chunk_size = 10000
reader = pd.read_csv('D:/Table/P5/DM-AS2/Test_data/best_feature_engineered_test_set_VU_DM.csv', chunksize=chunk_size)

predictions = []  # 创建一个空列表以存储每个块的预测结果
for chunk in reader:
    # 可以在这里添加数据预处理步骤，比如填充缺失值等
    chunk[features] = chunk[features].fillna(0)
    chunk_pred = model.predict(chunk[features])  # 应用模型进行预测
    chunk['predictions'] = chunk_pred  # 将预测结果添加到 DataFrame
    predictions.append(chunk[['srch_id', 'prop_id', 'predictions']])  # 仅保留需要的列

# 合并所有批次的预测结果
final_predictions = pd.concat(predictions)

# 确保按照预测分数排序，如果 Kaggle 要求
final_predictions.sort_values(['srch_id', 'predictions'], ascending=[True, False], inplace=True)

# 选择需要的列
final_predictions = final_predictions[['srch_id', 'prop_id']]

# 保存为 CSV 文件，确保不包含索引，包含列标题
final_predictions.to_csv('train=RF.csv', index=False, header=True)

print(final_predictions)